<a href="https://colab.research.google.com/github/arifuddinatif/face-emotion-recognition/blob/main/FERapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2

In [ ]:
# !pip install streamlit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install flask-ngrok

In [ ]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 5.2 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=c870c57ef4b88869aaaeb603be4caf0b2744b0d8ddd25835328856ab2e4845cc
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
! ngrok authtoken 1xFnYtGBLJtnVaSpGLwo9qhCOZu_4gRapCrF7aHizdUiCkR1r

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# %%writefile app.py
from flask import Flask,render_template,Response
from flask_ngrok import run_with_ngrok
import cv2
import numpy as np
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io


app=Flask(__name__,template_folder='/content/drive/MyDrive/Capstone Project 5/templates')
# run_with_ngrok(app)
# # import portpicker
# # port = portpicker.pick_unused_port()
# # from google.colab import output
# # output.serve_kernel_port_as_window(port)

# # from gevent.pywsgi import WSGIServer
# # host='localhost'
# # app_server = WSGIServer((host, port), app)
# # app_server.serve_forever()

# ## # JavaScript to properly create our live video stream using our webcam as input
# def video_stream():
#   js = Javascript('''
#     var video;
#     var div = null;
#     var stream;
#     var captureCanvas;
#     var imgElement;
#     var labelElement;
    
#     var pendingResolve = null;
#     var shutdown = false;
    
#     function removeDom() {
#        stream.getVideoTracks()[0].stop();
#        video.remove();
#        div.remove();
#        video = null;
#        div = null;
#        stream = null;
#        imgElement = null;
#        captureCanvas = null;
#        labelElement = null;
#     }
    
#     function onAnimationFrame() {
#       if (!shutdown) {
#         window.requestAnimationFrame(onAnimationFrame);
#       }
#       if (pendingResolve) {
#         var result = "";
#         if (!shutdown) {
#           captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
#           result = captureCanvas.toDataURL('image/jpeg', 0.8)
#         }
#         var lp = pendingResolve;
#         pendingResolve = null;
#         lp(result);
#       }
#     }
    
#     async function createDom() {
#       if (div !== null) {
#         return stream;
#       }

#       div = document.createElement('div');
#       div.style.border = '2px solid black';
#       div.style.padding = '3px';
#       div.style.width = '100%';
#       div.style.maxWidth = '600px';
#       document.body.appendChild(div);
      
#       const modelOut = document.createElement('div');
#       modelOut.innerHTML = "<span>Status:</span>";
#       labelElement = document.createElement('span');
#       labelElement.innerText = 'No data';
#       labelElement.style.fontWeight = 'bold';
#       modelOut.appendChild(labelElement);
#       div.appendChild(modelOut);
           
#       video = document.createElement('video');
#       video.style.display = 'block';
#       video.width = div.clientWidth - 6;
#       video.setAttribute('playsinline', '');
#       video.onclick = () => { shutdown = true; };
#       stream = await navigator.mediaDevices.getUserMedia(
#           {video: { facingMode: "environment"}});
#       div.appendChild(video);

#       imgElement = document.createElement('img');
#       imgElement.style.position = 'absolute';
#       imgElement.style.zIndex = 1;
#       imgElement.onclick = () => { shutdown = true; };
#       div.appendChild(imgElement);
      
#       const instruction = document.createElement('div');
#       instruction.innerHTML = 
#           '<span style="color: red; font-weight: bold;">' +
#           'When finished, click here or on the video to stop this demo</span>';
#       div.appendChild(instruction);
#       instruction.onclick = () => { shutdown = true; };
      
#       video.srcObject = stream;
#       await video.play();

#       captureCanvas = document.createElement('canvas');
#       captureCanvas.width = 640; //video.videoWidth;
#       captureCanvas.height = 480; //video.videoHeight;
#       window.requestAnimationFrame(onAnimationFrame);
      
#       return stream;
#     }
#     async function stream_frame(label, imgData) {
#       if (shutdown) {
#         removeDom();
#         shutdown = false;
#         return '';
#       }

#       var preCreate = Date.now();
#       stream = await createDom();
      
#       var preShow = Date.now();
#       if (label != "") {
#         labelElement.innerHTML = label;
#       }
            
#       if (imgData != "") {
#         var videoRect = video.getClientRects()[0];
#         imgElement.style.top = videoRect.top + "px";
#         imgElement.style.left = videoRect.left + "px";
#         imgElement.style.width = videoRect.width + "px";
#         imgElement.style.height = videoRect.height + "px";
#         imgElement.src = imgData;
#       }
      
#       var preCapture = Date.now();
#       var result = await new Promise(function(resolve, reject) {
#         pendingResolve = resolve;
#       });
#       shutdown = false;
      
#       return {'create': preShow - preCreate, 
#               'show': preCapture - preShow, 
#               'capture': Date.now() - preCapture,
#               'img': result};
#     }
#     ''')

#   display(js)

def video_frame(label, bbox):
    data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
    return data

def js_to_image(js_reply):
    image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

def bbox_to_bytes(bbox_array):
    # convert array into PIL image
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    # format bbox into png for return
    bbox_PIL.save(iobuf, format='png')
    # format return string
    bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
    return bbox_bytes

face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))



colour_cycle = ((255, 0, 0), (0, 255, 0), (0, 0, 255), (230, 230, 250))

from keras.models import model_from_json
import numpy as np
import cv2

def load_model(path):

	json_file = open(path + 'model.json', 'r')
	loaded_model_json = json_file.read()
	json_file.close()
	model = model_from_json(loaded_model_json)
	model.load_weights(path + "model.h5")
	print("Loaded model from disk")
	return model
	
def predict_emotion(gray, x, y, w, h):
	face = np.expand_dims(np.expand_dims(np.resize(gray[y:y+w, x:x+h]/255.0, (48, 48)),-1), 0)
	prediction = model.predict([face])
	return(int(np.argmax(prediction)), round(max(prediction[0])*100, 2))
 

path = "/content/drive/MyDrive/Capstone Project 5/"
model = load_model(path)

fcc_path = "Tools/haarcascade_frontalface_alt.xml"
faceCascade = cv2.CascadeClassifier(fcc_path)
emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: "Neutral"}
colour_cycle = ((255, 0, 0), (0, 255, 0), (0, 0, 255), (230, 230, 250))

cam = cv2.videoCapture() #pycharm entry

 
def gen_frames():
    label_html = 'Capturing...'
    count = 0

    while True:
        success,frame = camera.read()
        bbox = ''
        js_reply = video_frame(label_html,bbox)
        if not js_reply:
            print('Invalid')
        else:
            img = js_to_image(js_reply["img"])
            # create transparent overlay for bounding box
            bbox_array = np.zeros([480,640,4], dtype=np.uint8)
            # grayscale image for face detection
            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            #  get face region coordinates
            faces = face_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30, 30))
            # # get face bounding box for overlay
    # for (x,y,w,h) in faces:
    #   bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)

    # bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # # convert overlay of bbox into bytes
    # bbox_bytes = bbox_to_bytes(bbox_array)
    # # update bbox so next frame gets new overlay
    # bbox = bbox_bytes

            for (count,(x, y, w, h)) in enumerate(faces):   
                colour = colour_cycle[int(count%len(colour_cycle))]
                bbox_array = cv2.rectangle(bbox_array, (x, y), (x+w, y+h), colour, 2)
                bbox_array = cv2.line(bbox_array, (x+5, y+h+5),(x+100, y+h+5), colour, 20)
                bbox_array = cv2.putText(bbox_array, "Face #"+str(count+1), (x+5, y+h+11), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), lineType=cv2.LINE_AA)
                bbox_array = cv2.line(bbox_array, (x+8, y),(x+150, y), colour, 20)
                emotion_id, confidence = predict_emotion(gray, x, y, w, h)
                emotion = emotion_dict[emotion_id]
                bbox_array = cv2.putText(bbox_array, emotion + ": " + str(confidence) + "%" , (x+20, y+5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), lineType=cv2.LINE_AA)
      # out.write(img)
                bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
     # convert overlay of bbox into bytes
                bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
                bbox = bbox_bytes
                ret,buffer=cv2.imencode('.jpg',img)
                frame=buffer.tobytes()
      
      # cv2_imshow(img)
      # print("Frame number: " + str(counter))
      # counter = counter+1
      
            if cv2.waitKey(1) & 0xFF == ord('q'):
                print('Invalid')
@app.route('/')
def index():
    return render_template('index.html')
@app.route('/video_feed')
def video_feed():
    return Response(gen_frames(),mimetype='multipart/x-mixed-replace;boundary=frame')
if __name__ == '__main__':
  app.run()

In [ ]:
# %%writefile FERAPP.py
# import streamlit as st
# st.title("WebApp")
# run  = st.checkbox('Run')
# FRAME_WINDOW = st.image([])
# while run:
#   # import dependencies
#   from IPython.display import display, Javascript, Image
#   from google.colab.output import eval_js
#   from base64 import b64decode, b64encode
#   import cv2
#   import numpy as np
#   import PIL
#   import io
#   import html
#   import time


#   # function to convert the JavaScript object into an OpenCV image
#   def js_to_image(js_reply):
#   # decode base64 image
#     image_bytes = b64decode(js_reply.split(',')[1])
#   # convert bytes to numpy array
#     jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
#   # decode numpy array into OpenCV BGR image
#     img = cv2.imdecode(jpg_as_np, flags=1)

#     return img

# # function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
#   def bbox_to_bytes(bbox_array):
#     """
#     Params:
#             bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
#     Returns:
#           bytes: Base64 image byte string
#     """
#   # convert array into PIL image
#     bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
#     iobuf = io.BytesIO()
#   # format bbox into png for return
#     bbox_PIL.save(iobuf, format='png')
#   # format return string
#     bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

#     return bbox_bytes




# # initialize the Haar Cascade face detection model
#   face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))


  
#   def video_frame(label, bbox):
#     data = eval_js('stream_frame("{}", "{}")'.format(label,bbox))
#     return data
#   colour_cycle = ((255, 0, 0), (0, 255, 0), (0, 0, 255), (230, 230, 250))
#   from keras.models import model_from_json
#   import numpy as np
#   import cv2

#   def load_model(path):

# 	  json_file = open(path + 'model.json', 'r')
# 	  loaded_model_json = json_file.read()
# 	  json_file.close()
	
# 	  model = model_from_json(loaded_model_json)
# 	  model.load_weights(path + "model.h5")
# 	  print("Loaded model from disk")
# 	  return model
	
#   def predict_emotion(gray, x, y, w, h):
# 	  face = np.expand_dims(np.expand_dims(np.resize(gray[y:y+w, x:x+h]/255.0, (48, 48)),-1), 0)
# 	  prediction = model.predict([face])

# 	  return(int(np.argmax(prediction)), round(max(prediction[0])*100, 2))
   
#   path = "/content/drive/MyDrive/Capstone Project 5/"
#   model = load_model(path)

#   fcc_path = "Tools/haarcascade_frontalface_alt.xml"
#   faceCascade = cv2.CascadeClassifier(fcc_path)
#   emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: "Neutral"}
#   colour_cycle = ((255, 0, 0), (0, 255, 0), (0, 0, 255), (230, 230, 250))
#   from google.colab.patches import cv2_imshow
#   from IPython.display import clear_output

# # start streaming video from webcam
#   video_stream()
# # label for video
#   label_html = 'Capturing...'
# # initialze bounding box to empty
#   bbox = ''
#   count = 0 
#   counter = 1
  
#   while True:
#       js_reply = video_frame(label_html,bbox)
#       if not js_reply:
#           break

#     # convert JS response to OpenCV Image
#       img = js_to_image(js_reply["img"])

#     # create transparent overlay for bounding box
#       bbox_array = np.zeros([480,640,4], dtype=np.uint8)

#     # grayscale image for face detection
#       gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

#     # get face region coordinates
#       faces = face_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30, 30))
#     # # get face bounding box for overlay
#     # for (x,y,w,h) in faces:
#     #   bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)

#     # bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
#     # # convert overlay of bbox into bytes
#     # bbox_bytes = bbox_to_bytes(bbox_array)
#     # # update bbox so next frame gets new overlay
#     # bbox = bbox_bytes

#       for (count,(x, y, w, h)) in enumerate(faces):
#         colour = colour_cycle[int(count%len(colour_cycle))]
#         bbox_array = cv2.rectangle(bbox_array, (x, y), (x+w, y+h), colour, 2)
#         bbox_array = cv2.line(bbox_array, (x+5, y+h+5),(x+100, y+h+5), colour, 20)
#         bbox_array = cv2.putText(bbox_array, "Face #"+str(count+1), (x+5, y+h+11), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), lineType=cv2.LINE_AA)
#         bbox_array = cv2.line(bbox_array, (x+8, y),(x+150, y), colour, 20)
#         emotion_id, confidence = predict_emotion(gray, x, y, w, h)
#         emotion = emotion_dict[emotion_id]
#         bbox_array = cv2.putText(bbox_array, emotion + ": " + str(confidence) + "%" , (x+20, y+5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), lineType=cv2.LINE_AA)
#       # out.write(img)
#         bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
#      # convert overlay of bbox into bytes
#         bbox_bytes = bbox_to_bytes(bbox_array)
#     # update bbox so next frame gets new overlay
#         bbox = bbox_bytes
      
#       # cv2_imshow(img)
#       # print("Frame number: " + str(counter))
#       # counter = counter+1
      
#       if cv2.waitKey(1) & 0xFF == ord('q'):
#         break;
#   out.release()
# # webcam.release()
#   cv2.destroyAllWindows()
# else:
#   st.write('Stopped')

In [ ]:
# %%writefile app.py
# from flask import Flask, render_template, request
# import cv2
# from keras.models import load_model
# import numpy as np
# from flask_ngrok import run_with_ngrok
# app = Flask(__name__,template_folder='/content/drive/MyDrive/Capstone Project 5/templates')
# run_with_ngrok(app)
# import portpicker
# port = portpicker.pick_unused_port()
# from google.colab import output
# output.serve_kernel_port_as_window(port)

# from gevent.pywsgi import WSGIServer
# host='localhost'
# app_server = WSGIServer((host, port), app)
# app_server.serve_forever()

# app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 1


# @app.route('/')
# def index():
#     return render_template('index.html')

# @app.route('/after', methods=['GET', 'POST'])
# def after():
#     img = request.files['file1']

#     img.save('static/file.jpg')

#     ####################################
#     img1 = cv2.imread('static/file.jpg')
#     gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
#     cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')
#     faces = cascade.detectMultiScale(gray, 1.1, 3)

#     for x,y,w,h in faces:
#         cv2.rectangle(img1, (x,y), (x+w, y+h), (0,255,0), 2)

#         cropped = img1[y:y+h, x:x+w]

#     cv2.imwrite('static/after.jpg', img1)

#     try:
#         cv2.imwrite('static/cropped.jpg', cropped)

#     except:
#         pass

#     #####################################

#     try:
#         image = cv2.imread('static/cropped.jpg', 0)
#     except:
#         image = cv2.imread('static/file.jpg', 0)

#     image = cv2.resize(image, (48,48))

#     image = image/255.0

#     image = np.reshape(image, (1,48,48,1))

#     model = load_model('/content/drive/MyDrive/Capstone Project 5/model.h5')

#     prediction = model.predict(image)

#     label_map =   ['Anger','Neutral' , 'Fear', 'Happy', 'Sad', 'Surprise']

#     prediction = np.argmax(prediction)

#     final_prediction = label_map[prediction]

#     return render_template('After.html', data=final_prediction)

# if __name__ == "__main__":
#     app.run()

In [ ]:
# !nohup flask run --server.port 80 app.py &

nohup: appending output to 'nohup.out'


In [ ]:
# from pyngrok import ngrok

# url = ngrok.connect(port=8501)
# url

<NgrokTunnel: "http://e633-34-125-209-26.ngrok.io" -> "http://localhost:80">

In [ ]:
# ngrok.kill()

In [ ]:
!wget --no-check-certificate \
    https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/assets/haarcascade_frontalface_default.xml \
    -O haarcascade_frontalface_default.xml


# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time


# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes




# initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))


# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

--2021-09-09 07:45:44--  https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/assets/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.05s   

2021-09-09 07:45:44 (19.6 MB/s) - ‘haarcascade_frontalface_default.xml’ saved [930127/930127]



In [ ]:
colour_cycle = ((255, 0, 0), (0, 255, 0), (0, 0, 255), (230, 230, 250))


In [ ]:
from keras.models import model_from_json
import numpy as np
import cv2

def load_model(path):

	json_file = open(path + 'model.json', 'r')
	loaded_model_json = json_file.read()
	json_file.close()
	
	model = model_from_json(loaded_model_json)
	model.load_weights(path + "model.h5")
	print("Loaded model from disk")
	return model
	
def predict_emotion(gray, x, y, w, h):
	face = np.expand_dims(np.expand_dims(np.resize(gray[y:y+w, x:x+h]/255.0, (48, 48)),-1), 0)
	prediction = model.predict([face])

	return(int(np.argmax(prediction)), round(max(prediction[0])*100, 2))

In [ ]:
path = "/content/drive/MyDrive/Capstone Project 5/"
model = load_model(path)

fcc_path = "Tools/haarcascade_frontalface_alt.xml"
faceCascade = cv2.CascadeClassifier(fcc_path)
emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: "Neutral"}
colour_cycle = ((255, 0, 0), (0, 255, 0), (0, 0, 255), (230, 230, 250))


Loaded model from disk


In [ ]:
!wget --no-check-certificate \
    https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/assets/haarcascade_frontalface_default.xml \
    -O haarcascade_frontalface_default.xml

face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))


--2021-09-09 08:25:10--  https://raw.githubusercontent.com/computationalcore/introduction-to-opencv/master/assets/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.05s   

2021-09-09 08:25:10 (19.6 MB/s) - ‘haarcascade_frontalface_default.xml’ saved [930127/930127]



In [ ]:
import os

def convert_avi_to_mp4(avi_file_path, output_name):
    os.popen("ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{output}.mp4'".format(input = avi_file_path, output = output_name))
    return True

In [ ]:
cap = cv2.VideoCapture("/content/drive/MyDrive/Capstone Project 5/Videos/pexels-gabby-k-5273028.mp4")

while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.imwrite('kang'+str(i)+'.jpg',frame)
        i+=1
    import glob
    img_array = []
    for filename in glob.glob('/content/*.jpg'):
      img = cv2.imread(filename)
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)
    out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
    for i in range(len(img_array)):
      out.write(img_array[i])
    out.release()

    convert_avi_to_mp4('/content/sample_data/project.avi','project.mp4')
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
 
# Opens the Video file
cap= cv2.VideoCapture('/content/drive/MyDrive/Capstone Project 5/Videos/pexels-gabby-k-5273028.mp4')
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('kang'+str(i)+'.jpg',frame)
    i+=1
 
cap.release()
cv2.destroyAllWindows()